## Build Kaiser Window

In [1]:
import numpy as np
from scipy.special import i0
import matplotlib.pyplot as plt

# FIR Low-Pass Filter Design using Kaiser Window

This code designs and visualizes a digital FIR low-pass filter using the Kaiser window method.  
Given the desired ripple `delta`, transition width, and cutoff frequency, it:

- Computes filter order `N` based on attenuation and transition width.  
- Builds the Kaiser window.  
- Constructs the ideal low-pass kernel.  
- Multiplies with the window to obtain the practical filter.  
- Plots the window, ideal kernel, windowed kernel, and both frequency responses.


In [ ]:
delta = 0.001
transition_width = 0.1
cutoff = 0.2

A = -20 * np.log10(delta)
m = np.ceil((A - 8) / (2.285 * 2 * np.pi * transition_width))
beta = 0.1102 * (A - 8.7) if A > 50 else 0.5842 * (A - 21)**0.4 + 0.07886 * (A - 21) if A >= 21 else 0.0
m = int(m)
N = m + 1

n = np.arange(N)
alpha = (N - 1) / 2.0
w = i0(beta * np.sqrt(np.abs(1 - ((n - alpha) / alpha)**2))) / i0(beta)

h_ideal = 2 * cutoff * np.sinc(2 * cutoff * (n - alpha))
h_windowed = h_ideal * w

h_ideal_freq = np.fft.fft(h_ideal, 1024)
h_windowed_freq = np.fft.fft(h_windowed, 1024)
freq = np.fft.fftfreq(1024)

plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.stem(n, w)
plt.title('Kaiser Window w[n]')
plt.xlabel('n')
plt.ylabel('w[n]')

plt.subplot(2, 2, 2)
plt.stem(n, h_ideal)
plt.title('Ideal Low Pass Filter h[n]')
plt.xlabel('n')
plt.ylabel('h[n]')

plt.subplot(2, 2, 3)
plt.stem(n, h_windowed)
plt.title('Windowed Filter h[n]*w[n]')
plt.xlabel('n')
plt.ylabel('h[n]*w[n]')

plt.subplot(2, 2, 4)
plt.plot(freq[:512], 20*np.log10(np.abs(h_ideal_freq[:512])), 'r--', alpha=0.7, label='Ideal Filter')
plt.plot(freq[:512], 20*np.log10(np.abs(h_windowed_freq[:512])), 'b-', label='Windowed Filter')
plt.axhline(20*np.log10(delta), color='g', linestyle=':', label=f'Delta ({delta})')
plt.axvline(cutoff, color='k', linestyle=':', label='Cutoff')
plt.axvline(cutoff + transition_width/2, color='m', linestyle=':', label='Transition Band')
plt.axvline(cutoff - transition_width/2, color='m', linestyle=':')
plt.title('Frequency Response (dB)')
plt.xlabel('Normalized Frequency')
plt.ylabel('Magnitude (dB)')
plt.legend()
plt.ylim(-80, 5)
plt.grid(True)

plt.tight_layout()
plt.show()

### Notes
- The filter length `N` is automatically chosen to satisfy the given specifications.  
- `delta` controls the stopband ripple.  
- Transition width determines how sharp the cutoff is.  
- The Kaiser window parameter `beta` is adjusted depending on attenuation.  
- The plots illustrate how windowing shapes the filter in both time and frequency domain.


# FIR Differentiator Design using Kaiser Window

This code designs and analyzes an FIR digital differentiator using the Kaiser window method.  
Steps included:

- Derives the ideal differentiator impulse response \( h[n] = \frac{(-1)^n}{n}, \, h[0]=0 \).  
- Computes required filter length `N` based on desired ripple `delta` and transition width.  
- Generates a Kaiser window with parameter `beta`.  
- Forms the windowed kernel \( h[n] \cdot w[n] \).  
- Computes frequency responses for both the ideal and windowed filters.  
- Plots impulse responses and magnitude spectra.


In [ ]:

from scipy.signal import freqz, kaiser, kaiser_beta, kaiser_atten

# Ideal differentiator impulse response h[n] = (-1)^n / n, h[0] = 0
def ideal_diff(N):
    M = (N-1)//2
    n = np.arange(-M, M+1)
    h = np.zeros_like(n, dtype=float)
    h[n!=0] = ((-1)**abs(n[n!=0])) / n[n!=0]
    return n, h

# Parameters
delta = 0.001        # desired ripple
atten = -20*np.log10(delta)  # attenuation in dB
beta = kaiser_beta(atten)
tw = 0.5*np.pi        # transition width (rad)

N = int(np.ceil((atten - 8) / (2.285 * tw)))
if N % 2 == 0:
    N += 1
# Ideal kernel
n, h = ideal_diff(N)

# Kaiser window
w = kaiser(N, beta)

# Windowed kernel
h_win = h * w

# Frequency responses
wgrid, H = freqz(h, 1, 1024, whole=False)
_, H_win = freqz(h_win, 1, 1024, whole=False)

# Plots
plt.figure(figsize=(12,8))

plt.subplot(2,2,1)
plt.stem(n, h, basefmt=" ")
plt.title("Ideal h[n] = (-1)^n/n")
plt.xlabel("n")

plt.subplot(2,2,2)
plt.plot(wgrid, np.abs(H))
plt.title("Absolute value of Ideal H(e^jw)")
plt.xlabel("ω")

plt.subplot(2,2,3)
plt.stem(n, h_win, basefmt=" ")
plt.title("Windowed h[n]*w[n] (Kaiser)")
plt.xlabel("n")

plt.subplot(2,2,4)
plt.plot(wgrid, np.abs(H_win))
plt.title("Absolute value of Windowed H(e^jw)")
plt.xlabel("ω")

plt.tight_layout()
plt.show()


### Notes
- Filter length `N` is adjusted to be odd, ensuring an antisymmetric (Type-III) FIR differentiator.  
- The Kaiser window parameter `beta` is automatically set from attenuation.  
- The unwindowed kernel is infinite-length; truncation plus windowing yields the practical filter.  
- The magnitude plots compare the ideal differentiator response with the windowed approximation.  
